In [202]:
from sklearn.neighbors import KNeighborsClassifier
import csv
import pickle
import re
from datetime import datetime
import numpy as np
# randn = np.random.randn
from pandas import *
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.stats import linregress
from pylab import *
from urllib import urlopen
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from __future__ import division

## Variables Pre-processing

1 - emp.var.rate = 1-value

2 - euribor = 1/value

3 - job = 1 if student or retired, 0 if other

4 - marital(?) = 1 if single, o if other

5 - poutcome(?) = 1 if success 0 if other

6 - age = ((exp^((value-47)/3))-(exp^-((value-47)/3))/2 (lowest 36-58 highest 19 and 77)

7 - campaign = 1/value

8 - pdays = 1 if < 999 0 if = 999 (as "contacted before?")

9 - previous = value


math.exp(x) = e ** x

math.expm1(x) = e ** -x

In [ ]:
#Hyperbola centered around y axis to describe age influence (too steep)
#ex = math.exp((x-47)/3)
#emx = math.exp(-(x-47)/3)
#y = ((ex)+(emx))/2
#age = (y-1)/5000

In [255]:
bank_full = read_csv('bank-additional-full.csv', delimiter=';', header=0)
bank_full.head(3)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [256]:
bank = bank_full[['age','job','marital','campaign','pdays','previous','poutcome','emp.var.rate','euribor3m','y']]
bank.head(3)

,age,job,marital,campaign,pdays,previous,poutcome,emp.var.rate,euribor3m,y
0,56,housemaid,married,1,999,0,nonexistent,1.1,4.857,no
1,57,services,married,1,999,0,nonexistent,1.1,4.857,no
2,37,services,married,1,999,0,nonexistent,1.1,4.857,no


In [257]:
bank['y'] = bank['y'].map({'yes': 1, 'no': 0})
bank['y']

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
...
41173    1
41174    1
41175    0
41176    0
41177    0
41178    1
41179    0
41180    0
41181    1
41182    0
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, Length: 41188, dtype: int64

In [258]:
bank['age']

0     56
1     57
2     37
3     40
4     56
5     45
6     59
7     41
8     24
9     25
10    41
11    25
12    29
13    57
14    35
...
41173    62
41174    62
41175    34
41176    38
41177    57
41178    62
41179    64
41180    36
41181    37
41182    29
41183    73
41184    46
41185    56
41186    44
41187    74
Name: age, Length: 41188, dtype: int64

In [259]:
bank['age'] = abs((bank['age']-47)/30)
bank['age'][bank['age'] > 1] = 1
bank['age']

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


0     0.300000
1     0.333333
2     0.333333
3     0.233333
4     0.300000
5     0.066667
6     0.400000
7     0.200000
8     0.766667
9     0.733333
10    0.200000
11    0.733333
12    0.600000
13    0.333333
14    0.400000
...
41173    0.500000
41174    0.500000
41175    0.433333
41176    0.300000
41177    0.333333
41178    0.500000
41179    0.566667
41180    0.366667
41181    0.333333
41182    0.600000
41183    0.866667
41184    0.033333
41185    0.300000
41186    0.100000
41187    0.900000
Name: age, Length: 41188, dtype: float64